In [49]:
import json
# Sample JSON file
json_file = "main_DComp_backup"+ ".json"
# change bellow to
                    #else:
                     #   dod[18]["val"][0] = "Miron:2025:rep:"
                     #   dod[19]["val"][0] = "PSINa25"
def read_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def process_json(data):
    for item in data:
        print(f"Processing item with key: {item.get('key', [])}")
        for entry in item.get("dod", []):
            print(f"ID: {entry.get('id', 'N/A')}, Label: {entry.get('label', 'Unknown')}, Value: {entry.get('val', 'None')}")

In [50]:
# Read and process JSON data
json_data = read_json(json_file)
#process_json(json_data)

In [51]:
import pandas as pd
def load_references(file_path):
    """Load reference data from Excel file."""
    df = pd.read_excel(file_path, sheet_name="Sheet1")
    ref_dict = dict(zip(df["Ref_abb"].str.replace(" ", ""), df["Ref_GEMS"]))  # Remove spaces for comparison
    return ref_dict

In [52]:
# Load reference data
reference_file = "references.xlsx"
reference_data = load_references(reference_file)

In [53]:
def process_items_18_19(data, references):
    """Process item 18 and update item 19 based on reference mapping."""
    for item in data:
        dod = item.get("dod", [])
        if len(dod) > 18:  # Ensure items 18 and 19 exist
            val_18 = dod[18].get("val", [])  # Item 18 (Index 17)
            if isinstance(val_18, list):
                updated_val_19 = []
                updated_val_18 = []
                for value in val_18:
                    if ":" in value:
                        # Case 1: If ':' exists, split and process both parts
                        first_part, second_part = value.split(":", 1)
                        second_part = second_part.replace(" ", "")  # Remove spaces
                        second_partb = second_part
                        second_part = references.get(second_part, second_part)  # Replace if match exists
                        updated_val_19.append(first_part)
                        if second_part == second_partb:
                            updated_val_18.append(f"{value}")
                        else:
                            updated_val_18.append(f"{second_part}")
                    else:
                        # Case 2: If no ':', check if value matches Ref_abb
                        cleaned_value = value.replace(" ", "")  # Remove spaces for matching
                        new_value = references.get(cleaned_value, value)  # Replace if match exists
                        updated_val_19.append(value)
                        updated_val_18.append(new_value)

                if len(dod) > 19:  # Ensure item 19 exists
                    dod[19]["dM"] = 1  # Set dM to 1 for item 19
                    dod[19]["val"] = updated_val_19  # Updated list

                dod[18]["val"] = updated_val_18  # Updated item 18
                dod[19]["dN"] = len(dod[19]["val"])
                dod[18]["dN"] = len(dod[18]["val"])
                if len(dod[19]["val"]) > 1:
                    if dod[19]["val"][1] == "": 
                        dod[18]["val"][0] = "Miron:2025:rep:"
                        dod[19]["val"][0] = "PSINa25"
                    else:
                        dod[18]["val"][0] = "Miron:2024:rep:" # for RDC "Miron:2025:rep:"
                        dod[19]["val"][0] = "PSINa25"
    return data

In [54]:
processed_data = process_items_18_19(json_data, reference_data)

In [55]:
#processed_data

In [56]:
def update_vm_ref_values(data):
    """Update item 18 with 'Miron:2024:rep' and item 19 with 'Vm_Ref' if 'Vm_Ref:' is present in item 18."""
    for item in data:
        dod = item.get("dod", [])
        if len(dod) > 19:  # Ensure items 18 and 19 exist
            val_18 = dod[18].get("val", [])  # Item 18
            val_19 = dod[19].get("val", [])  # Item 19

            if isinstance(val_18, list) and isinstance(val_19, list):
                updated_val_18 = []
                updated_val_19 = val_19[:]  # Preserve original values from item 19

                for value in val_18:
                    if "Vm_Ref:" in value:
                        updated_val_19 = []
                        updated_val_18.append("Hummel_ea:2023:dat:")  # Modify item 18
                        updated_val_18.append("Miron:2024:rep:")  # Modify item 18
                        updated_val_19.append("TDB2020") 
                        updated_val_19.append("Vm_Ref") 

                        dod[19]["dN"] = 2  
                        dod[18]["dN"] = 2  
                    else:
                        updated_val_18.append(value)  # Preserve item 18 values

                dod[19]["dM"] = 1  # Set dM to 1 for item 19
                dod[19]["val"] = updated_val_19  # Update item 19
                dod[18]["val"] = updated_val_18  # Update item 18

    return data

In [57]:
processed_data = update_vm_ref_values(processed_data)

In [58]:
#processed_data

In [59]:
def save_json(data, filename="formatted_references.json"):
    """Save formatted data to a JSON file."""
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [60]:
# Save formatted data to a JSON file
save_json(processed_data, "formatted_"+json_file)